In [1]:
from diffusers.utils import load_image

from models.combined_pipeline import CombinedPipeline

In [ ]:
pipe = CombinedPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-decoder-inpaint')
pipe = pipe.to('cuda')
pipe.enable_model_cpu_offload()
pipe.enable_xformers_memory_efficient_attention()

unet\diffusion_pytorch_model.safetensors not found


Loading pipeline components...: 0it [00:00, ?it/s]

prior\diffusion_pytorch_model.safetensors not found


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

C:\Users\Alexey\anaconda3\envs\hands-tracking\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alexey\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
from diffusers.utils import make_image_grid

img_url = "https://raw.githubusercontent.com/Fantasy-Studio/Paint-by-Example/main/examples/image/example_1.png"
mask_url = "https://raw.githubusercontent.com/Fantasy-Studio/Paint-by-Example/main/examples/mask/example_1.png"
example_url = "https://raw.githubusercontent.com/Fantasy-Studio/Paint-by-Example/main/examples/reference/example_1.jpg"

dog_image = load_image(img_url).resize((512, 512))
mask_image = load_image(mask_url).resize((512, 512))
panda_image = load_image(example_url).resize((512, 512))

make_image_grid([dog_image, mask_image, panda_image], 1, 3)

In [ ]:
pipe(dog_image,
     mask_image,
     mask_image,
     guidance_scale=0.0)